In [2]:
import pandas as pd
import numpy as np
import ast
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv(r"../output/3060.csv")

In [4]:
# List of metric names
metric_names = ["f1","mse","accuracy","recall","precision", "auc","area_under_pr", "r2_score", "rmse"]
dataset = df.copy()
# Create columns based on metric names
# Convert string representations to dictionaries
dataset['output_metrics'] = dataset['output_metrics'].apply(ast.literal_eval)
# Create columns based on metric names
for metric in metric_names:
    dataset[metric] = dataset['output_metrics'].apply(lambda x: x.get(metric) if isinstance(x, dict) else np.nan)

In [5]:
# Specify the metric you want to maximize
metric_to_maximize = 'accuracy'
tomax = {"f1": True,
         "mse": False,
         "accuracy": True,
         "recall": True,
         "precision": True,
         "auc": True,
         "area_under_pr": True,
        "rmse": False,
        "r2_score": True}


# Filter out rows with NaN values in the specified metric column
filtered_df = dataset.dropna(subset=[metric_to_maximize]).loc[dataset[metric_to_maximize] != "nan"]

# Find the rows that maximize the specified metric for each dataset
if  tomax[metric_to_maximize]:
    max_rows = filtered_df.sort_values(["dataset",metric_to_maximize], ascending = False)
else:
    max_rows = filtered_df.sort_values(["dataset",metric_to_maximize], ascending = True)
    
sel_cols = ['dataset', 'model', 'run_time',
       'eval_metric', 'best_params', 'output_metrics',
 'debug_preds', 'f1', 'mse', 'accuracy',
       'recall', 'precision', 'auc', 'area_under_pr', 'r2_score', 'rmse']

max_rows[sel_cols].head(50)

,dataset,model,run_time,eval_metric,best_params,output_metrics,debug_preds,f1,mse,accuracy,recall,precision,auc,area_under_pr,r2_score,rmse
27,heloc,catboost,299.412001,roc_auc,"{'bagging_temperature': 0.8106712891382661, 'd...","{'recall': 0.7759764185703758, 'precision': 0....","[1, 1, 1, 1, 1, 0, 1, 0, 0, 1]",0.741549,NaN,0.719312,0.775976,0.710047,0.795409,0.667232,NaN,NaN
22,heloc,fttransformer,351.359779,roc_auc,"{'AdamW_weight_decay': 0.009146661940655405, '...","{'recall': 0.7862932940309506, 'precision': 0....","[1, 1, 1, 1, 1, 0, 0, 0, 0, 1]",0.742778,NaN,0.717400,0.786293,0.703826,0.786277,0.664312,NaN,NaN
23,heloc,categoryembedding,122.302854,roc_auc,"{'AdamW_weight_decay': 0.0016439896907364268, ...","{'recall': 0.7612380250552689, 'precision': 0....","[1, 1, 1, 0, 1, 0, 1, 0, 0, 1]",0.733925,NaN,0.713576,0.761238,0.708505,0.781335,0.663241,NaN,NaN
25,heloc,autoint,117.289257,roc_auc,"{'AdamW_weight_decay': 0.00712395199162431, 'A...","{'recall': 0.7243920412675019, 'precision': 0....","[1, 1, 1, 0, 1, 0, 0, 0, 0, 0]",0.722263,NaN,0.710899,0.724392,0.720147,0.784077,0.664689,NaN,NaN
24,heloc,gandalf,136.814297,roc_auc,"{'AdamW_weight_decay': 0.009462603733172785, '...","{'recall': 0.7391304347826086, 'precision': 0....","[1, 1, 1, 0, 1, 0, 0, 0, 0, 1]",0.723926,NaN,0.707457,0.739130,0.709335,0.780707,0.659664,NaN,NaN
26,heloc,tabtransformer,192.353009,roc_auc,"{'AdamW_weight_decay': 0.009179016413844592, '...","{'recall': 0.7192336035372144, 'precision': 0....","[1, 0, 1, 0, 1, 0, 0, 0, 0, 1]",0.714495,NaN,0.701721,0.719234,0.709818,0.769133,0.656223,NaN,NaN
20,heloc,s1dcnn,485.250577,roc_auc,"{'batch_size': 1442, 'hidden_size': 2048, 'lea...","{'recall': 0.5637435519528371, 'precision': 0....","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0.653288,NaN,0.689484,0.563744,0.776650,0.775765,0.664218,NaN,NaN
21,heloc,tabnet,50.608528,roc_auc,"{'AdamW_weight_decay': 0.000936224711762532, '...","{'recall': 0.5689019896831246, 'precision': 0....","[1, 1, 1, 0, 1, 1, 0, 0, 0, 0]",0.595909,NaN,0.599618,0.568902,0.625608,0.627471,0.579619,NaN,NaN
28,covertype,s1dcnn,4925.375239,accuracy,"{'batch_size': 3990, 'hidden_size': 4096, 'lea...","{'accuracy': 0.9629818317005501, 'f1': 0.96316...","[0, 1, 1, 0, 1, 0, 2, 5, 1, 0]",0.963165,NaN,0.962982,NaN,NaN,NaN,NaN,NaN,NaN
19,adult,catboost,966.330410,roc_auc,"{'bagging_temperature': 0.2847041279241091, 'd...","{'recall': 0.6551020408163265, 'precision': 0....","[0, 1, 0, 0, 1, 0, 1, 0, 0, 0]",0.716318,NaN,0.875077,0.655102,0.790154,0.930202,0.600668,NaN,NaN


In [6]:
maximized_rows["best_params"].iloc[0]

"{'Adam_weight_decay': 0.0008344870704054452, 'ReduceLROnPlateau_factor': 0.39054754693784877, 'ReduceLROnPlateau_patience': 9, 'batch_size': 64, 'cat_emb_dim': 3, 'clip_value': 2, 'early_stopping_patience': 5, 'gamma': 1.1112005798198572, 'lambda_sparse': 0.0018702554034918367, 'learning_rate': 0.00023539065760010136, 'mask_type': 'entmax', 'momentum': 0.002024407113236184, 'n_d': 11, 'n_independent': 2, 'n_shared': 3, 'n_steps': 6, 'optimizer_fn': <class 'torch.optim.adam.Adam'>, 'scheduler_fn': 'ReduceLROnPlateau', 'virtual_batch_size_ratio': 0.5, 'weights': 1, 'outer_params': {'max_epochs': 1000, 'early_stopping_tolerance': 1e-06, 'hyperopt_evals': 10, 'val_size': 0.2, 'auto_lr_find': False}, 'virtual_batch_size': 32, 'optimizer_params': {'weight_decay': 0.0008344870704054452}, 'scheduler_params': {'factor': 0.39054754693784877, 'patience': 9, 'min_lr': 1e-07, 'verbose': True, 'mode': 'min'}, 'n_a': 11}"

In [17]:
maximized_rows["best_params"].iloc[1]

"{'alpha': 1, 'colsample_bytree': 0.7701468147287692, 'gamma': 4, 'lambda': 5, 'learning_rate': 0.019628488779340834, 'max_bin': 156, 'max_depth': 8, 'min_child_weight': 8, 'n_estimators': 255, 'subsample': 0.9280869759290384, 'tree_method': 'auto'}"